In [5]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage.transform import resize

from keras.utils import to_categorical

from keras.applications.vgg19 import preprocess_input

from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, GlobalAveragePooling2D, Dropout

In [6]:
# To stop potential randomness
seed = 42
rng = np.random.RandomState(seed)

In [7]:
data = pd.read_csv('emergency/train.csv')

In [8]:
data.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [9]:
X = []
for img_name in data.image_names:
    img = plt.imread('emergency/images/' + img_name)
    X.append(img)
X = np.array(X)

In [11]:
y = data['emergency_or_not'].values
y = to_categorical(y)

In [12]:
X.min(),X.max()

(0, 255)

In [13]:
X = preprocess_input(X)
X=np.tanh(X)

In [14]:
X.shape

(1646, 224, 224, 3)

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
base_model = VGG19(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

80142336/80134624 [==============================] - 48s 1us/step


In [17]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [18]:
base_model_pred = base_model.predict(X_train)
base_model_pred_valid = base_model.predict(X_valid)

In [19]:
base_model_pred.shape

(1316, 7, 7, 512)

In [20]:
base_model_pred = base_model_pred.reshape(1316, 7*7*512)

In [21]:
base_model_pred.shape

(1316, 25088)

In [23]:
base_model_pred_valid.shape

(330, 7, 7, 512)

In [24]:
base_model_pred_valid = base_model_pred_valid.reshape(330, 7*7*512)

In [25]:
base_model_pred_valid.shape

(330, 25088)

In [26]:
base_model_pred.min(), base_model_pred.max()

(0.0, 10.839293)

In [27]:
max_val = base_model_pred.max()

In [28]:
base_model_pred = base_model_pred / max_val
base_model_pred_valid = base_model_pred_valid / max_val
base_model_pred.min(), base_model_pred.max()

(0.0, 1.0)

In [29]:
model = Sequential()
model.add(InputLayer((7*7*512, )))
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [30]:
model.compile(optimizer='sgd', metrics=['accuracy'], loss='categorical_crossentropy')

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 130       
Total params: 26,388,546
Trainable params: 26,388,546
Non-trainable params: 0
____________________________________________

In [36]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [37]:
final_weights_path = 'best_cnn_model.h5'
callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', patience=5, verbose=1)
]

In [41]:
history = model.fit(base_model_pred, y_train, batch_size=18,epochs=200,validation_data=(base_model_pred_valid, y_valid), callbacks=callbacks_list, shuffle=False, workers=5, use_multiprocessing=True)

Epoch 1/200
73/74 [============================>.] - ETA: 0s - loss: 0.0773 - accuracy: 0.9871
Epoch 00001: val_loss improved from inf to 1.01184, saving model to best_cnn_model.h5
74/74 [==============================] - 1s 19ms/step - loss: 0.0772 - accuracy: 0.9871 - val_loss: 1.0118 - val_accuracy: 0.7333
Epoch 2/200
73/74 [============================>.] - ETA: 0s - loss: 0.0882 - accuracy: 0.9779
Epoch 00002: val_loss improved from 1.01184 to 0.97600, saving model to best_cnn_model.h5
74/74 [==============================] - 1s 19ms/step - loss: 0.0883 - accuracy: 0.9780 - val_loss: 0.9760 - val_accuracy: 0.7394
Epoch 3/200
73/74 [============================>.] - ETA: 0s - loss: 0.0145 - accuracy: 0.9962
Epoch 00003: val_loss improved from 0.97600 to 0.47045, saving model to best_cnn_model.h5
74/74 [==============================] - 1s 19ms/step - loss: 0.0145 - accuracy: 0.9962 - val_loss: 0.4704 - val_accuracy: 0.8667
Epoch 4/200
73/74 [============================>.] - ETA: 0

In [42]:
data = pd.read_csv('emergency/test.csv')
X = []
for img_name in data.image_names:
    img = plt.imread('emergency/images/' + img_name)
    X.append(img)
X = np.array(X)
X = preprocess_input(X)
X=np.tanh(X)
base_model_pred = base_model.predict(X)
base_model_pred.shape

(706, 7, 7, 512)

In [43]:
base_model_pred = base_model_pred.reshape(706, 7*7*512)
max_val = base_model_pred.max()
base_model_pred = base_model_pred / max_val
base_model_pred.min(),base_model_pred.max()

(0.0, 1.0)

In [44]:
predictions = model.predict_classes(base_model_pred)
predictions

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,